In [0]:
########## Run this code snippet when running for the first time and don't repeat it in future (else it will keep on downloading the same stuffs again and again and
########## result in redundant usage of memory)

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apachemirror.wuchna.com/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
import findspark
import pandas as pd
import random
import string
import numpy as np
os.environ["JAVA_HOME"]   = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]  = "/content/spark-2.4.4-bin-hadoop2.7"
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark                      = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
pdf = pd.DataFrame(columns=['Index','strings'])
pdf['Index']  = [100,101,102,103]
pdf['strings'] = [''.join([str(elem) for elem in random.choices(string.ascii_letters,k=8)]) for j in range(4)]
pdf['left_space'] = ['   '+''.join([str(elem) for elem in random.choices(string.ascii_letters,k=8)]) for j in range(4)]
pdf['right_space'] = [''.join([str(elem) for elem in random.choices(string.ascii_letters,k=8)])+'    ' for j in range(4)]
pdf['delim_string'] = ['.'.join([str(elem) for elem in random.choices(string.ascii_letters,k=8)]) for j in range(4)]
pdf['alpha_numerics'] = [''.join([str(elem) for elem in random.choices(string.ascii_letters+string.digits,k=10)]) for j in range(4)]
df = spark.createDataFrame(pdf)
df.show(truncate=False)

+-----+--------+-----------+------------+---------------+--------------+
|Index|strings |left_space |right_space |delim_string   |alpha_numerics|
+-----+--------+-----------+------------+---------------+--------------+
|100  |uoNFkLox|   HAJxbNfv|iPtTsupf    |E.o.W.I.E.g.G.i|tOVruNgNig    |
|101  |EiiFPRoQ|   oXcyEsgR|vtGhiunc    |o.v.D.z.V.l.F.Y|o3w9CereQ4    |
|102  |NDaNERFA|   htyNjYjZ|XDJGjVnv    |k.a.r.Y.a.r.r.C|W4X8aYFMdg    |
|103  |ceKGKzgn|   TitMuPIS|HvlcJtGC    |z.a.z.Q.v.Y.r.v|JAbosZ4vKd    |
+-----+--------+-----------+------------+---------------+--------------+



In [0]:
df = df.withColumn('lower_string',lower(df['strings']))
df = df.withColumn('upper_string',upper(df['strings']))
df = df.withColumn('initialized_with_cap',initcap(df['strings']))
df = df.withColumn('length_string',length(df['strings']))
df = df.withColumn('ascii_string_first_val',ascii(df['strings']))
df = df.withColumn('substring',substring(df['strings'],pos = 2,len =3))
df = df.withColumn('substring_index_positive',substring_index(df['delim_string'],delim='.',count=3))
df = df.withColumn('substring_index_negative',substring_index(df['delim_string'],delim='.',count=-2))
df.show()

+-----+--------+-----------+------------+---------------+--------------+------------+------------+--------------------+-------------+----------------------+---------+------------------------+------------------------+
|Index| strings| left_space| right_space|   delim_string|alpha_numerics|lower_string|upper_string|initialized_with_cap|length_string|ascii_string_first_val|substring|substring_index_positive|substring_index_negative|
+-----+--------+-----------+------------+---------------+--------------+------------+------------+--------------------+-------------+----------------------+---------+------------------------+------------------------+
|  100|uoNFkLox|   HAJxbNfv|iPtTsupf    |E.o.W.I.E.g.G.i|    tOVruNgNig|    uonfklox|    UONFKLOX|            Uonfklox|            8|                   117|      oNF|                   E.o.W|                     G.i|
|  101|EiiFPRoQ|   oXcyEsgR|vtGhiunc    |o.v.D.z.V.l.F.Y|    o3w9CereQ4|    eiifproq|    EIIFPROQ|            Eiifproq|            8

In [0]:
print('Splitting with respect to a particular condition. Here we are splitting a string on occurance of alpha-numeric value')
df = df.withColumn('split',split(df['alpha_numerics'], '[0-9]+'))
df.select('alpha_numerics','split').show()

Splitting with respect to a particular condition. Here we are splitting a string on occurance of alpha-numeric value
+--------------+---------------+
|alpha_numerics|          split|
+--------------+---------------+
|    tOVruNgNig|   [tOVruNgNig]|
|    o3w9CereQ4|[o, w, CereQ, ]|
|    W4X8aYFMdg| [W, X, aYFMdg]|
|    JAbosZ4vKd|  [JAbosZ, vKd]|
+--------------+---------------+



In [0]:
print('We will concatenate two columns of df- alpha_numerics and lower_string')
df = df.withColumn('Concatenated_string',concat(df['alpha_numerics'],df['lower_string']))
df.select('Concatenated_string','lower_string','alpha_numerics').show()

We will concatenate two columns of df- alpha_numerics and lower_string
+-------------------+------------+--------------+
|Concatenated_string|lower_string|alpha_numerics|
+-------------------+------------+--------------+
| tOVruNgNiguonfklox|    uonfklox|    tOVruNgNig|
| o3w9CereQ4eiifproq|    eiifproq|    o3w9CereQ4|
| W4X8aYFMdgndanerfa|    ndanerfa|    W4X8aYFMdg|
| JAbosZ4vKdcekgkzgn|    cekgkzgn|    JAbosZ4vKd|
+-------------------+------------+--------------+



In [0]:
print('We will concatenate two columns of df- alpha_numerics and lower_string separated by a comma')
df = df.withColumn('Concatenated_string_sep',concat_ws(',',df['alpha_numerics'],df['lower_string']))
df.select('Concatenated_string_sep','lower_string','alpha_numerics').show()

We will concatenate two columns of df- alpha_numerics and lower_string separated by a comma
+-----------------------+------------+--------------+
|Concatenated_string_sep|lower_string|alpha_numerics|
+-----------------------+------------+--------------+
|    tOVruNgNig,uonfklox|    uonfklox|    tOVruNgNig|
|    o3w9CereQ4,eiifproq|    eiifproq|    o3w9CereQ4|
|    W4X8aYFMdg,ndanerfa|    ndanerfa|    W4X8aYFMdg|
|    JAbosZ4vKd,cekgkzgn|    cekgkzgn|    JAbosZ4vKd|
+-----------------------+------------+--------------+



In [0]:
print('We will right pad a string column with 0 and left pad with 1. The size of actual string column is 8. We will make it 11 with extra space filled with padding')
df = df.withColumn('right_paded',rpad(df['lower_string'],11,'0'))
df = df.withColumn('left_paded',lpad(df['lower_string'],11,'1'))
df.select('left_paded','lower_string','right_paded').show(truncate=False)

We will right pad a string column with 0 and left pad with 1. The size of actual string column is 8. We will make it 11 with extra space filled with padding
+-----------+------------+-----------+
|left_paded |lower_string|right_paded|
+-----------+------------+-----------+
|111uonfklox|uonfklox    |uonfklox000|
|111eiifproq|eiifproq    |eiifproq000|
|111ndanerfa|ndanerfa    |ndanerfa000|
|111cekgkzgn|cekgkzgn    |cekgkzgn000|
+-----------+------------+-----------+



In [0]:
print('We will trim the extra spaces from left(using ltrim) and right(using rtrim) of a string in a column.')
df = df.withColumn('right_space_removed',rtrim(df['right_space']))
df = df.withColumn('left_space_removed',ltrim(df['left_space']))
df = df.withColumn('len_space_trim_left',length(df['left_space_removed']))
df = df.withColumn('len_with_space_left',length(df['left_space']))
df.select('right_space_removed','len_space_trim_left','len_with_space_left','left_space_removed','left_space','right_space').show(truncate=False)
print('we can however remove spaces from both the ends of a string consecutively using trim(it performs ltrim and rtrim together)')

We will trim the extra spaces from left(using ltrim) and right(using rtrim) of a string in a column.
+-------------------+-------------------+-------------------+------------------+-----------+------------+
|right_space_removed|len_space_trim_left|len_with_space_left|left_space_removed|left_space |right_space |
+-------------------+-------------------+-------------------+------------------+-----------+------------+
|iPtTsupf           |8                  |11                 |HAJxbNfv          |   HAJxbNfv|iPtTsupf    |
|vtGhiunc           |8                  |11                 |oXcyEsgR          |   oXcyEsgR|vtGhiunc    |
|XDJGjVnv           |8                  |11                 |htyNjYjZ          |   htyNjYjZ|XDJGjVnv    |
|HvlcJtGC           |8                  |11                 |TitMuPIS          |   TitMuPIS|HvlcJtGC    |
+-------------------+-------------------+-------------------+------------------+-----------+------------+

we can however remove spaces from both the ends of

In [0]:
print('We will find the occurance of a phase for the first time in string.')
df = df.withColumn('occurance_first_time',instr(df['strings'],'Q'))
df.select('occurance_first_time','strings').show()

We will find the occurance of a phase for the first time in string.
+--------------------+--------+
|occurance_first_time| strings|
+--------------------+--------+
|                   0|uoNFkLox|
|                   8|EiiFPRoQ|
|                   0|NDaNERFA|
|                   0|ceKGKzgn|
+--------------------+--------+



In [0]:
print('We will find the reversed string.')
df = df.withColumn('reversed',reverse(df['strings']))
df.select('reversed','strings').show()

We will find the reversed string.
+--------+--------+
|reversed| strings|
+--------+--------+
|xoLkFNou|uoNFkLox|
|QoRPFiiE|EiiFPRoQ|
|AFRENaDN|NDaNERFA|
|ngzKGKec|ceKGKzgn|
+--------+--------+

